In [1]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset("penguins")
# Esborram les files que contenen elements NA
df_pingus = df.dropna()
df_pingus.head()  

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male


In [2]:
##
## REPRESENTACIÓ EN VALORS NUMÈRICS LES DIFERENTS ESPÈCIES
##

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_pingus['species_encoded'] = le.fit_transform(df_pingus['species'])



/tmp/ipykernel_11325/3445530508.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pingus['species_encoded'] = le.fit_transform(df_pingus['species'])


In [3]:
##
## DIVIDIM EL DATASET EN DUES PARTS
## UN 80% DADES D'ENTRENAMENT I UN 20% DADES DE PROVA
##

from sklearn.model_selection import train_test_split

# 3. Separar X i Y
X = df_pingus.drop(['species', 'species_encoded'], axis=1) # Totes les altres variables
Y = df_pingus['species_encoded']

# 4. Separar conjunts d'entrenament i prova
# Usem una proporció habitual, p. ex., 80% entrenament / 20% prova
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y
)




In [4]:
numerical_cols = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
categorical_cols = ['island', 'sex']

In [5]:
from sklearn.feature_extraction import DictVectorizer

# Convertir el DataFrame (subconjunt) a llista de diccionaris (per a DictVectorizer)
X_train_cat_dict = X_train[categorical_cols].to_dict('records')
X_test_cat_dict = X_test[categorical_cols].to_dict('records')

# Inicialitzar i aplicar DictVectorizer (només fit a l'entrenament)
dict_vectorizer = DictVectorizer(sparse=False) # sparse=False per obtenir un array dens
X_train_cat_encoded = dict_vectorizer.fit_transform(X_train_cat_dict)
X_test_cat_encoded = dict_vectorizer.transform(X_test_cat_dict)

print("Columnes generades pel DictVectorizer:")
print(dict_vectorizer.get_feature_names_out())

Columnes generades pel DictVectorizer:
['island=Biscoe' 'island=Dream' 'island=Torgersen' 'sex=Female' 'sex=Male']


In [6]:
from sklearn.preprocessing import StandardScaler

# Inicialitzar l'escalador
standard_scaler = StandardScaler()

# Aplicar l'escalat (fit i transform a l'entrenament, només transform a la prova)
X_train_num_scaled = standard_scaler.fit_transform(X_train[numerical_cols])
X_test_num_scaled = standard_scaler.transform(X_test[numerical_cols])

# Opcional: Comprovar que la mitjana és ~0 i la desviació ~1 (al conjunt d'entrenament)
print("\nMitjana de bill_length_mm (Entrenament):", X_train_num_scaled[:, 0].mean()) # Hauria de ser prop de 0
print("Desviació estàndard de bill_length_mm (Entrenament):", X_train_num_scaled[:, 0].std()) # Hauria de ser prop de 1


Mitjana de bill_length_mm (Entrenament): 8.815003864692972e-16
Desviació estàndard de bill_length_mm (Entrenament): 1.0


In [7]:
import numpy as np

# Concatenar les dades codificades (categòriques) i escalades (numèriques)
X_train_processed = np.hstack([X_train_cat_encoded, X_train_num_scaled])
X_test_processed = np.hstack([X_test_cat_encoded, X_test_num_scaled])

print("\nForma final de X_train processat:", X_train_processed.shape)
print("Forma final de X_test processat:", X_test_processed.shape)


Forma final de X_train processat: (266, 9)
Forma final de X_test processat: (67, 9)


In [8]:
# Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

import pickle

models = {
    'log_reg': LogisticRegression(max_iter=1000, random_state=42),
    'svm': SVC(kernel='linear', random_state=42),
    'dt': DecisionTreeClassifier(random_state=42),
    'knn': KNeighborsClassifier(n_neighbors=5)
}

trained_models = {}
for name, model in models.items():
    print(f"Entrenant el model: {name}")
    model.fit(X_train_processed, Y_train)
    trained_models[name] = model
    score = model.score(X_test_processed, Y_test)
    print(f"Precisió en el conjunt de prova: {score:.4f}\n")

# --- 6. SERIALITZACIÓ (GUARDAR COMPONENTS) ---
# Guardem el preprocessor, l'encoder de Y i els 4 models.
with open('../models/dict_vectorizer.pkl', 'wb') as f:
    pickle.dump(dict_vectorizer, f)

with open('../models/standard_scaler.pkl', 'wb') as f:
    pickle.dump(standard_scaler, f)
    
with open('../models/label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

for name, model in trained_models.items():
    with open(f'../models/{name}_model.pkl', 'wb') as f:
        pickle.dump(model, f)

print("\n✅ Tots els models i components de preprocessament s'han guardat amb èxit.")
print("Fitxers generats: dict_vectorizer.pkl, standard_scaler.pkl, label_encoder.pkl i els 4 models.")

Entrenant el model: log_reg
Precisió en el conjunt de prova: 0.9851

Entrenant el model: svm
Precisió en el conjunt de prova: 0.9851

Entrenant el model: dt
Precisió en el conjunt de prova: 0.9254

Entrenant el model: knn
Precisió en el conjunt de prova: 0.9851


✅ Tots els models i components de preprocessament s'han guardat amb èxit.
Fitxers generats: dict_vectorizer.pkl, standard_scaler.pkl, label_encoder.pkl i els 4 models.
